In [1]:
import sys
print(sys.path)

from langchain.llms  import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.utilities import SQLDatabase

import pandas as pd
import os
import sys
import json
import ast
import sqlite3

# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

['/home/llmuser', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/llmuser/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages']


In [2]:
#Make sure the model path is correct for your system!
local_generation = LlamaCpp(
    model_path="models/sqlcoder-7b-2.fp16.gguf",
    temperature=0.0,
    n_gpu_layers=-1,
    n_ctx=1024,
    top_p=1,
    callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)


ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 6 CUDA devices:
  Device 0: NVIDIA GeForce RTX 3060 Ti, compute capability 8.6, VMM: yes
  Device 1: NVIDIA GeForce RTX 3060 Ti, compute capability 8.6, VMM: yes
  Device 2: NVIDIA GeForce RTX 2060, compute capability 7.5, VMM: yes
  Device 3: NVIDIA GeForce RTX 2060, compute capability 7.5, VMM: yes
  Device 4: NVIDIA GeForce RTX 2060, compute capability 7.5, VMM: yes
  Device 5: NVIDIA GeForce RTX 2060, compute capability 7.5, VMM: yes
llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from models/sqlcoder-7b-2.fp16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = hub
llama_model_l

0:3:0


Model metadata: {'tokenizer.ggml.add_eos_token': 'false', 'tokenizer.ggml.unknown_token_id': '0', 'tokenizer.ggml.eos_token_id': '2', 'general.architecture': 'llama', 'llama.rope.freq_base': '1000000.000000', 'llama.context_length': '16384', 'general.name': 'hub', 'tokenizer.ggml.add_bos_token': 'true', 'llama.embedding_length': '4096', 'llama.feed_forward_length': '11008', 'llama.attention.layer_norm_rms_epsilon': '0.000010', 'llama.rope.dimension_count': '128', 'tokenizer.ggml.bos_token_id': '1', 'llama.attention.head_count': '32', 'llama.block_count': '32', 'llama.attention.head_count_kv': '32', 'tokenizer.ggml.model': 'llama', 'general.file_type': '1'}


In [3]:
OUTPUT_DIR = '/home/llmuser/models/results/'

DATASET_SCHEMA = '/home/llmuser/models/datasets/spider-2/tables.json'
DATASET = '/home/llmuser/models/datasets/spider-2/dev.json'
SPIDER_DB_DIR = '/home/llmuser/models/datasets/spider-2/database/' #the path were all databases exist

def load_data_json(DATASET):
    # Open the JSON file and load data
    with open(DATASET, 'r') as json_file:
        return json.load(json_file)


In [4]:


# Modificar esto eventualmente por una tecnica de RAG
def getSQLDatabase(db_id):
    # Modificar el PATH para donde este la base de datos
    db_path = SPIDER_DB_DIR
    uri = 'sqlite:///' + db_path + db_id + '/' + db_id + '.sqlite'
    db = SQLDatabase.from_uri(uri,  sample_rows_in_table_info=0)
    return db


def string_to_array(input):
    return ast.literal_eval(input)

def correctCase(llm_table_list, db_SQLDatabase):
    corrected_tables = []
    db_table_list = db_SQLDatabase.get_usable_table_names()
    for table_name in llm_table_list:
        for table in db_table_list:
            if table_name.casefold() == table.casefold():
                corrected_tables.append(table)
    return corrected_tables

def getPertinentSchema(llm_table_list, db_SQLDatabase):
    return db_SQLDatabase.get_table_info(llm_table_list)


def saveDataJson(OUTPUT_PATH,list):
    # Save the object to a JSON file
    with open(OUTPUT_PATH, 'w') as json_file:
        json.dump(list, json_file)

def load_data_json(DATASET):
    # Open the JSON file and load data
    with open(DATASET, 'r') as json_file:
        return json.load(json_file)

In [5]:
prompt_temp = """### Task
        Generate a SQL for MySQL query to answer [QUESTION]{question}[/QUESTION]
        
        ### Database Schema
        This query will run on a database whose schema is represented in this string:
        {schema}
        
        ### Answer
        Given the database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION]
        [SQL]
        """

SQLite_PROMPT = PromptTemplate(
    input_variables=["question", "schema"],
    template= prompt_temp
)



In [ ]:
INPUT_FILE1 = '/home/llmuser/models/results/gpt/tables-gpt4-v2.json'

if __name__ == '__main__':
    CODEX = load_data_json(INPUT_FILE1)
    for index, row in enumerate(CODEX):
        print("")
        print(f"index is {index}")
        #print(row)
        if not row['error_encountered']:
            db_SQLDatabase = getSQLDatabase(row['db_id'])
            error_encountered = False
            error = None
            try:
                llm_table_list = string_to_array(row['table_list'])
                corrected_llm_table_list = correctCase(llm_table_list, db_SQLDatabase)
            except Exception as e:
                error = e
                error_encountered = True
            if not error_encountered:
                pertinent_schema = getPertinentSchema(corrected_llm_table_list, db_SQLDatabase)
                if pertinent_schema:
                    try:  
                        prompt = SQLite_PROMPT.format(question = row['question'], schema = pertinent_schema)
                        #print(prompt)
                        predicted_query = local_generation(prompt)
                    except Exception as e:
                        error = e
                        error_encountered = True
            if not error_encountered:
                row.update({'predicted_query': predicted_query})
            else:
                row['error_encountered'] = True
                row['error_type'] = type(error).__name__
                row['error_message'] = str(error)
            
    OUTPUT_DIR = '/home/llmuser/models/results/gpt/'
    OUTPUT_FILE_DIFF = 'tables-gpt4-sqlcoder-7b.json'
    OUTPUT_PATH = os.path.join(OUTPUT_DIR, OUTPUT_FILE_DIFF)
    saveDataJson(OUTPUT_PATH, CODEX)


index is 0


Llama.generate: prefix-match hit


 SELECT COUNT(s.Singer_ID) FROM singer s;
index is 1



llama_print_timings:        load time =     577.18 ms
llama_print_timings:      sample time =       7.05 ms /    15 runs   (    0.47 ms per token,  2128.87 tokens per second)
llama_print_timings: prompt eval time =    1339.34 ms /   182 tokens (    7.36 ms per token,   135.89 tokens per second)
llama_print_timings:        eval time =     625.83 ms /    14 runs   (   44.70 ms per token,    22.37 tokens per second)
llama_print_timings:       total time =    2060.47 ms /   196 tokens
Llama.generate: prefix-match hit


 SELECT COUNT(s.Singer_ID) AS total_singers FROM singer s;
index is 2


...


llama_print_timings:        load time =     577.18 ms
llama_print_timings:      sample time =      12.85 ms /    27 runs   (    0.48 ms per token,  2101.49 tokens per second)
llama_print_timings: prompt eval time =    1475.00 ms /   215 tokens (    6.86 ms per token,   145.76 tokens per second)
llama_print_timings:        eval time =    1158.53 ms /    26 runs   (   44.56 ms per token,    22.44 tokens per second)
llama_print_timings:       total time =    2792.17 ms /   241 tokens
Llama.generate: prefix-match hit


 SELECT tc.package_option, tc.series_name FROM "TV_Channel" tc WHERE tc.hight_definition_TV = 'Yes';
index is 632



llama_print_timings:        load time =     577.18 ms
llama_print_timings:      sample time =      17.59 ms /    37 runs   (    0.48 ms per token,  2103.35 tokens per second)
llama_print_timings: prompt eval time =    1525.91 ms /   220 tokens (    6.94 ms per token,   144.18 tokens per second)
llama_print_timings:        eval time =    1607.59 ms /    36 runs   (   44.66 ms per token,    22.39 tokens per second)
llama_print_timings:       total time =    3336.50 ms /   256 tokens
Llama.generate: prefix-match hit


 SELECT tc.package_option, tc.series_name FROM "TV_Channel" tc WHERE tc.hight_definition_TV = 'Yes';
index is 633



llama_print_timings:        load time =     577.18 ms
llama_print_timings:      sample time =      17.07 ms /    37 runs   (    0.46 ms per token,  2167.80 tokens per second)
llama_print_timings: prompt eval time =    1583.52 ms /   230 tokens (    6.88 ms per token,   145.25 tokens per second)
llama_print_timings:        eval time =    1623.10 ms /    36 runs   (   45.09 ms per token,    22.18 tokens per second)
llama_print_timings:       total time =    3406.26 ms /   266 tokens
Llama.generate: prefix-match hit


In [17]:

def getdbPath(db_id):
    # Modificar el PATH para donde este la base de datos
    db_path = SPIDER_DB_DIR
    db_uri = db_path + db_id + '/' + db_id + '.sqlite'
    return db_uri

def queryDB(database, SQL_query):
    conn = sqlite3.connect(database)
    # Create a cursor object to execute SQL queries
    cursor = conn.cursor()
    # Create a table
    cursor.execute("PRAGMA case_sensitive_like = OFF")
    cursor.execute(SQL_query)
    results = cursor.fetchall()
    cursor.close()
    conn.close()
    return results

def correctQuery(CODEX):
    for row in CODEX:
        if not row['error_encountered']:
            row['predicted_query'] = row['predicted_query'].replace('ILIKE', 'LIKE', -1)
            row['predicted_query'] = row['predicted_query'].replace('[/SQL]', '', -1)

def evaluatePredict(CODEX):
    count = 0
    for row in CODEX:
        predict_query_valid = True
        if not row['error_encountered']:
            db = getdbPath(row['db_id'])
            error_encountered = False
            error = None
            error_type = None
            try:
                goldQueryValue =  queryDB(db, row['gold_query'])
            except Exception as e:
                error_encountered = True
                error_type = 'Cannot_Query_Gold'
                error_message = str(e)
                predict_query_valid = False
                goldQueryValue = None
            pred_query = row['predicted_query']
            if not error_encountered:
                try:
                    predQueryValue = queryDB(db, pred_query)
                except Exception as e:
                    error_encountered = True
                    error_type = 'Cannot_Query_Pred'
                    error_message = str(e)
                    predQueryValue = None
            if not error_encountered:
                if goldQueryValue == predQueryValue :
                    predict_success = True
                else:
                    predict_success = False
                    error_encountered = True
                    error_type = 'Gold_Predict_Missmatch'
                    error_message = None
            row.update({'predict_success':predict_success, 'gold_result':goldQueryValue, 'predict_result': predQueryValue,'predict_query_valid':predict_query_valid})
            if error_encountered:
                row['error_encountered'] = error_encountered
                row['error_type'] = error_type
                row['error_message'] = error_message
    return CODEX

OUTPUT_DIR = '/home/llmuser/models/results/gpt/'
OUTPUT_FILE_DIFF = 'tables-gpt4-sqlcoder-7b.json'
OUTPUT_PATH = os.path.join(OUTPUT_DIR, OUTPUT_FILE_DIFF)

CODEX = load_data_json(OUTPUT_PATH)

correctQuery(CODEX)
CODEX_OUT = evaluatePredict(CODEX)

In [18]:
def saveDataJson(OUTPUT_PATH,list):
    # Save the object to a JSON file
    with open(OUTPUT_PATH, 'w') as json_file:
        json.dump(list, json_file)

OUTPUT_DIR = '/home/llmuser/models/results/gpt/'
OUTPUT_FILE_DIFF = 'gpt4-sqlcoder-7b-results-2.json'

OUTPUT_PATH = os.path.join(OUTPUT_DIR, OUTPUT_FILE_DIFF)

saveDataJson(OUTPUT_PATH, CODEX_OUT)